# Classifications
https://desktop.arcgis.com/en/arcmap/10.3/manage-data/las-dataset/lidar-point-classification.htm


In [45]:
import pandas as pd

classes_esri = pd.read_html("https://desktop.arcgis.com/en/arcmap/10.3/manage-data/las-dataset/lidar-point-classification.htm")
classes_esri[1]

,Classification value,Meaning
0,0,Never classified
1,1,Unassigned
2,2,Ground
3,3,Low Vegetation
4,4,Medium Vegetation
5,5,High Vegetation
6,6,Building
7,7,Low Point
8,8,Reserved*
9,9,Water


If we are starting with LAZ files we can convert to LAS if we wish with this function.

In [55]:
def convertLAZtoLAS():

    import sys
    import traceback
    import laspy
    import os
    try:
        print('Running LAZ_to_LAS.py')
        
        def convert_laz_to_las(in_laz, out_las):
            las = laspy.read(in_laz)
            las = laspy.convert(las)
            las.write(out_las)        
        
        in_dir = './'
        
        for (dirpath, dirnames, filenames) in os.walk(in_dir):
            for inFile in filenames:
                if inFile.endswith('.laz'):	
                    in_laz = os.path.join(dirpath,inFile)
                    
                    out_las = in_laz.replace('laz', 'las') 
                    print('working on file: ',out_las)
                    convert_laz_to_las(in_laz, out_las)
                                
        print('Finished without errors - LAZ_to_LAS.py')
    except:
        tb = sys.exc_info()[2]
        tbinfo = traceback.format_tb(tb)[0]
        print('Error in read_xmp.py')
        print ("PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1]))  




Now we can run this function to convert all of our LAZ files to LAS.

In [56]:
convertLAZtoLAS()

Running LAZ_to_LAS.py
working on file:  ./classification_17.las
working on file:  ./classification_2.las
working on file:  ./classification_3.las
working on file:  ./classification_4.las
working on file:  ./classification_5.las
working on file:  ./classification_7.las
working on file:  ./jogging_bridge_br.las
working on file:  ./classification_9.las
working on file:  ./classification_18.las
Finished without errors - LAZ_to_LAS.py


: 

In [1]:
import open3d as o3d
import laspy
import numpy as np


In [2]:
las = laspy.read('bearriver_downtown.laz')

In [5]:
print(las)

<LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 36455780 points, 3 vlrs)>


In [3]:
type(las)

laspy.lasdata.LasData

In [16]:
#list(las.point_format.dimension_names)

Let's view all of the pre assigned classifications for this dataset.

In [4]:
classifications = list(set(las.classification))

type(classifications)
print(classifications)

[2, 3, 4, 5, 7, 9, 17, 18]


In [7]:
def get_classifications_laspy(classifications,las):

    #create dictionary with complete data for each province
    d = {"original_dataset": las}
    #d = {}
    print("The classifications contained in this .las dataset are: ")
    print(classifications)

    for classification in classifications:
        print("Saving classification number: " +str(classification))
        classified_points = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
        classified_points.points = las.points[las.classification == classification]
        
        d["classification_{}".format(str(classification))] = classified_points
    
    return d

def laspy_classifications_to_laz(classifications_laspy):


    for classification_las in classifications_laspy:

        if classification_las == "original_dataset":
            pass
            print("skipped the original dataset.")

        else:
            print("processing {} to a LAZ file: ".format(classification_las))
            print(classifications_laspy[classification_las])
            classifications_laspy[classification_las].write("classification_{}.laz".format(classification_las.split("_")[1]))
            print("done")

def visualize_las(las):

    dataset = np.vstack((las.X,las.Y,las.Z)).T
    geom = o3d.geometry.PointCloud()
    geom.points = o3d.utility.Vector3dVector(dataset)
    o3d.visualization.draw_geometries([geom])

In [8]:

classifications = list(set(las.classification))
classifications_laspy =  get_classifications_laspy(classifications=classifications,las=las)
#laspy_classifications_to_laz(classifications_laspy=classifications_laspy)




The classifications contained in this .las dataset are: 
[2, 3, 4, 5, 7, 9, 17, 18]
Saving classification number: 2
Saving classification number: 3
Saving classification number: 4
Saving classification number: 5
Saving classification number: 7
Saving classification number: 9
Saving classification number: 17
Saving classification number: 18


In [9]:
classifications_laspy

{'original_dataset': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 36455780 points, 3 vlrs)>,
 'classification_2': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 2943578 points, 0 vlrs)>,
 'classification_3': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 7716819 points, 0 vlrs)>,
 'classification_4': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 324350 points, 0 vlrs)>,
 'classification_5': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 24392200 points, 0 vlrs)>,
 'classification_7': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 7322 points, 0 vlrs)>,
 'classification_9': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 1059170 points, 0 vlrs)>,
 'classification_17': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 12057 points, 0 vlrs)>,
 'classification_18': <LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 284 points,

In [14]:
visualize_las(classifications_laspy["classification_9"])


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 

In [10]:
import pandas as pd

classes_esri = pd.read_html("https://desktop.arcgis.com/en/arcmap/10.3/manage-data/las-dataset/lidar-point-classification.htm")
classes_esri[1]



,Classification value,Meaning
0,0,Never classified
1,1,Unassigned
2,2,Ground
3,3,Low Vegetation
4,4,Medium Vegetation
5,5,High Vegetation
6,6,Building
7,7,Low Point
8,8,Reserved*
9,9,Water
